In [1]:
# prepare the key for openai and pubmed
import os

# E2B sandbox API
with open("e2b.key", "r") as f:
    E2B_API_KEY = f.read()
    os.environ["E2B_API_KEY"] = E2B_API_KEY

# OpenAI API key
with open("openai.key", "r") as f:
    OPENAI_API_KEY_GLOBAL = f.read()
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY_GLOBAL

#  Pubmed API key
with open("pubmed.key", "r") as f:
    DEFAULT_PUBMED_API_KEY = f.read()
    os.environ["PUBMED_API_KEY"] = DEFAULT_PUBMED_API_KEY

# Screening criteria generation

In [2]:
# test the apis
from trialmind.api import ScreeningCriteriaGeneration
api = ScreeningCriteriaGeneration()
ecs = api.run(
    population = "Patients with cancer undergoing treatment with immune checkpoint inhibitors (ICIs)",
    intervention = "The occurrence of immune-related adverse events (irAEs)",
    comparator = "Patients who did not develop irAEs",
    outcome = "Overall survival (OS) and progression-free survival (PFS)",
    llm = "openai-gpt-4o",
    num_title_criteria=3,
    num_abstract_criteria=3
)

In [3]:
ecs['title_criteria']

['Does the title mention immune checkpoint inhibitors (ICIs)?',
 'Does the title refer to cancer patients or oncology?',
 'Does the title indicate a focus on survival outcomes (e.g., overall survival, progression-free survival)?']

In [4]:
ecs['eligibility_analysis']

['Studies must include patients diagnosed with cancer and undergoing treatment with immune checkpoint inhibitors (ICIs). Rationale: This ensures the population studied is relevant to the research question.',
 'Studies must report on the occurrence of immune-related adverse events (irAEs). Rationale: This is the primary intervention of interest and necessary for comparison.',
 'Studies must compare outcomes between patients who developed irAEs and those who did not. Rationale: This allows for the direct comparison needed to assess the impact of irAEs on survival outcomes.',
 'Studies must report on overall survival (OS) and/or progression-free survival (PFS). Rationale: These are the primary outcomes of interest and are essential for the meta-analysis.',
 'Studies must be peer-reviewed and published in English. Rationale: Ensures the quality and accessibility of the studies included in the meta-analysis.']

# Run screening

In [7]:
from trialmind.pubmed import pmid2papers
df_papers = pmid2papers(['38860089',
 '38883368',
 '38863868',
 '38865375',
 '38870979',
 '38899854',
 '38862251',
 '38893255',
 '38900577',
 '38901176',
 '38900018',
 '38870745',
 '38905806',
 '38877461',
 '38859854',
 '38896941',
 '38901174',
 '38898429',
 '38856777',
 '38885605',
 '38907273',
 '38886117',
 '38910480',
 '38860746',
 '38906102',
 '38883673',
 '38911862',
 '38881932',
 '38890622',
 '38857909',
 '38878765',
 '38872482',
 '38880924',
 '38871690',
 '38885362',
 '38870061',
 '38870013',
 '38909166',
 '38898592',
 '38883775',
 '38885840',
 '38913219',
 '38881944',
 '38862242',
 '38911372',
 '38859608',
 '38864258',
 '38881915',
 '38900575',
 '38882039',
 '38911075',
 '38871738',
 '38863432',
 '38896790',
 '38865481',
 '38857914',
 '38905818',
 '38912939',
 '38910605',
 '38877773',
 '38887231',
 '38892599',
 '38868282',
 '38863767',
 '38882997',
 '38886156',
 '38902698',
 '38859698',
 '38881937',
 '38902261',
 '38900646',
 '38884220',
 '38897136',
 '38888683',
 '38883687',
 '38873823',
 '38907159',
 '38908859',
 '38862908',
 '38911988',
 '38893277',
 '38890503',
 '38903931',
 '38872851',
 '38908836',
 '38912586',
 '38909011',
 '38907174',
 '38874014',
 '38889288',
 '38908858',
 '38886133',
 '38884085',
 '38896129',
 '38866824',
 '38886756',
 '38888461',
 '38911864',
 '38890738',
 '38867255'],
api_key=os.getenv("PUBMED_API_KEY", None)
)

In [8]:
from trialmind.api import LiteratureScreening
api = LiteratureScreening()

papers = df_papers[0]["Title"] + ": " + df_papers[0]["Abstract"].fillna("") # important to fillna
papers = papers.tolist()

In [9]:
ec_pred = api.run(
    population = "Patients with cancer undergoing treatment with immune checkpoint inhibitors (ICIs)",
    intervention = "The occurrence of immune-related adverse events (irAEs)",
    comparator = "Patients who did not develop irAEs",
    outcome = "Overall survival (OS) and progression-free survival (PFS)",
    llm = "openai-gpt-4o",
    criteria = ecs['title_criteria'] + ecs['content_criteria'],
    papers = papers[:100], # make for the top-100 for demo
)
ec_pred

[['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['YES', 'NO', 'NO', 'YES', 'NO', 'UNCERTAIN'],
 ['YES', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['YES', 'YES', 'NO', 'YES', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'YES', 'NO', 'UNCERTAIN'],
 ['YES', 'YES', 'NO', 'YES', 'YES', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'YES', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['YES', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'NO', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['NO', 'YES', 'NO', 'NO', 'NO', 'UNCERTAIN'],
 ['YES', 'YES', 'NO', 'YES', 'YES', 'UNCERTAIN'],
 